In [40]:
from google.colab import files
uploaded = files.upload()

Saving exp-data.csv to exp-data (1).csv


In [41]:
import pandas as pd
import io

df = pd.read_csv(io.StringIO(uploaded['exp-data.csv'].decode('utf-8')))
df

,timestamp,device_id,type,value
0,1493569701,photon_001,temperature,21.81
1,1493569701,photon_001,sleep,24346.00
2,1493569701,photon_001,battery,100.00
3,1493569711,photon_002,temperature,20.83
4,1493569711,photon_002,sleep,300000.00
...,...,...,...,...
125794,1499756486,photon_001,sleep,56930.00
125795,1499756486,photon_001,battery,86.00
125796,1499756574,photon_001,temperature,35.94
125797,1499756574,photon_001,sleep,56759.00


In [42]:
from datetime import datetime, timedelta, timezone
df['timestamp'] = df['timestamp'].apply(lambda timestamp: datetime.fromtimestamp(timestamp, tz=timezone(timedelta(hours=2)))) # Unix timestamp -> datetime
df.loc[df['type'] == "sleep", ['value']] = df[df['type'] == "sleep"]['value'].apply(lambda time: time / (60 * 1e3)) # Milliseconds -> minutes
df['hour'] = df['timestamp'].apply(lambda time: time.hour) # Extract hour of day
df['day'] = df['timestamp'].apply(lambda time: time.date()) # datetime -> date
df

,timestamp,device_id,type,value,hour,day
0,2017-04-30 18:28:21+02:00,photon_001,temperature,21.810000,18,2017-04-30
1,2017-04-30 18:28:21+02:00,photon_001,sleep,0.405767,18,2017-04-30
2,2017-04-30 18:28:21+02:00,photon_001,battery,100.000000,18,2017-04-30
3,2017-04-30 18:28:31+02:00,photon_002,temperature,20.830000,18,2017-04-30
4,2017-04-30 18:28:31+02:00,photon_002,sleep,5.000000,18,2017-04-30
...,...,...,...,...,...,...
125794,2017-07-11 09:01:26+02:00,photon_001,sleep,0.948833,9,2017-07-11
125795,2017-07-11 09:01:26+02:00,photon_001,battery,86.000000,9,2017-07-11
125796,2017-07-11 09:02:54+02:00,photon_001,temperature,35.940000,9,2017-07-11
125797,2017-07-11 09:02:54+02:00,photon_001,sleep,0.945983,9,2017-07-11


In [43]:
df[['device_id', 'type', 'value']].groupby(['device_id', 'type']).describe()

value             ...                      
                          count       mean  ...       75%         max
device_id  type                             ...                      
photon_001 battery      34182.0  81.673805  ...  87.00000  100.000000
           sleep        34182.0   2.516633  ...   2.66385  195.271433
           temperature  34182.0  30.204808  ...  34.53000   37.490000
photon_002 battery       7751.0  54.049081  ...  87.00000  100.000000
           sleep         7751.0   5.000000  ...   5.00000    5.000000
           temperature   7751.0  25.800852  ...  29.80000   37.300000

[6 rows x 8 columns]

In [44]:
exp_interval = df[['device_id', 'timestamp']].groupby(['device_id']).describe().copy()
exp_interval['timestamp', 'run_days'] = exp_interval['timestamp'].apply(lambda row: row['last'] - row['first'], axis=1)
exp_interval

timestamp         ...                                           
               count unique  ...                      last         run_days
device_id                    ...                                           
photon_001    102546  34182  ... 2017-07-11 09:02:54+02:00 71 days 14:34:33
photon_002     23253   7751  ... 2017-06-03 05:39:09+02:00 33 days 11:10:38

[2 rows x 7 columns]

In [45]:
import plotly.graph_objects as go
import plotly.express as px

In [84]:
def update_device_names(fig):
  fig.update_traces(name="Device w/ Hypnos", selector=dict(name="device_id=photon_001"))
  fig.update_traces(name="Device w/o Hypnos", selector=dict(name="device_id=photon_002"))

def style_legend(fig):
  fig.update_layout(
      legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
      ),
      font=dict(size=16)
  )

def remove_year(fig):
  fig.update_xaxes(
    tickformatstops=[
                     dict(dtickrange=[None, None], value="%e %b")
    ]
  )

In [47]:
df[df['type'] == 'battery'][['device_id', 'value']].groupby(['device_id']).describe()

value                                                     
              count       mean        std   min   25%   50%   75%    max
device_id                                                               
photon_001  34182.0  81.673805   7.320917  45.0  77.0  83.0  87.0  100.0
photon_002   7751.0  54.049081  34.786473  -5.0  16.0  59.0  87.0  100.0

In [86]:
fig1 = px.scatter(df[df['type'] == 'battery'], x='timestamp', y='value', color='device_id', trendline='lowess')

fig1.update_layout(
    xaxis_title="Date", 
    yaxis_title="Remaining battery (%)")

fig1.update_traces(
    marker=dict(size=3),
    line=dict(width=4, color="lightslategrey"))

update_device_names(fig1)
remove_year(fig1)

style_legend(fig1)
fig1.update_layout(legend= {'itemsizing': 'constant'})

fig1.show()

Output hidden; open in https://colab.research.google.com to view.

In [49]:
df[df['type'] == 'sleep'][['device_id', 'value']].groupby('device_id').describe()

value                      ...                             
              count      mean       std  ...     50%      75%         max
device_id                                ...                             
photon_001  34182.0  2.516633  3.943602  ...  1.3305  2.66385  195.271433
photon_002   7751.0  5.000000  0.000000  ...  5.0000  5.00000    5.000000

[2 rows x 8 columns]

In [85]:
fig2 = px.line(df[df['type'] == 'sleep'], x='timestamp', y='value', color='device_id')

fig2.update_layout(
    xaxis_title="Date", 
    yaxis_title="Sleep time (minutes)")

update_device_names(fig2)
remove_year(fig2)

style_legend(fig2)

fig2.show()

In [51]:
last_wo_hypnos_ts = df[df['device_id'] == 'photon_002']['timestamp'].max()
sbs_df = df[df['timestamp'] <= last_wo_hypnos_ts]
sbs_df

,timestamp,device_id,type,value,hour,day
0,2017-04-30 18:28:21+02:00,photon_001,temperature,21.810000,18,2017-04-30
1,2017-04-30 18:28:21+02:00,photon_001,sleep,0.405767,18,2017-04-30
2,2017-04-30 18:28:21+02:00,photon_001,battery,100.000000,18,2017-04-30
3,2017-04-30 18:28:31+02:00,photon_002,temperature,20.830000,18,2017-04-30
4,2017-04-30 18:28:31+02:00,photon_002,sleep,5.000000,18,2017-04-30
...,...,...,...,...,...,...
75466,2017-06-03 05:33:17+02:00,photon_001,sleep,20.551400,5,2017-06-03
75467,2017-06-03 05:33:17+02:00,photon_001,battery,62.000000,5,2017-06-03
75468,2017-06-03 05:39:09+02:00,photon_002,temperature,28.470000,5,2017-06-03
75469,2017-06-03 05:39:09+02:00,photon_002,sleep,5.000000,5,2017-06-03


In [52]:
sbs_df[sbs_df['type'] == 'battery'][['device_id', 'value']].groupby('device_id').describe()

value                                                     
              count       mean        std   min   25%   50%   75%    max
device_id                                                               
photon_001  17406.0  82.377226   7.374012  61.0  78.0  83.0  88.0  100.0
photon_002   7751.0  54.049081  34.786473  -5.0  16.0  59.0  87.0  100.0

In [53]:
sbs_bat_obs = sbs_df[sbs_df['type'] == 'battery']
d1_obs = sbs_bat_obs[sbs_bat_obs['device_id'] == 'photon_001']['value'].count()
d2_obs = sbs_bat_obs[sbs_bat_obs['device_id'] == 'photon_002']['value'].count()
print('Device w/ Hypnos sent {} observations'.format(d1_obs))
print('Device w/o Hypnos sent {} observations'.format(d2_obs))
print('Device w/ Hypnos sent {:.2f}x more observations during the same time period!'.format(d1_obs / d2_obs))

Device w/ Hypnos sent 17406 observations
Device w/o Hypnos sent 7751 observations
Device w/ Hypnos sent 2.25x more observations during the same time period!


In [87]:
import plotly.figure_factory as ff

fig5 = ff.create_distplot([
                           sbs_bat_obs[sbs_bat_obs['device_id'] == 'photon_001']['value'],
                           sbs_bat_obs[sbs_bat_obs['device_id'] == 'photon_002']['value']
], ['Device w/ Hypnos', 'Device w/o Hypnos'], colors=px.colors.qualitative.Plotly)

fig5.update_layout(
    xaxis_title='Battery (%)', 
    yaxis_title='Probability')

style_legend(fig5)
fig5.update_layout(legend=dict(
    traceorder='normal'
))

fig5.show()

In [55]:
hourly_stats = sbs_bat_obs[['device_id', 'hour', 'value']].groupby(['device_id', 'hour']).describe()
hourly_stats

value                        ...                    
                  count       mean        std  ...   50%    75%    max
device_id  hour                                ...                    
photon_001 0      324.0  72.277778   4.785491  ...  71.0  72.25   85.0
           1      285.0  70.940351   4.528465  ...  70.0  71.00   83.0
           2      237.0  69.696203   4.273552  ...  69.0  70.00   81.0
           3      207.0  68.541063   4.122753  ...  68.0  69.00   80.0
           4      175.0  67.451429   3.922626  ...  67.0  68.00   78.0
           5      155.0  66.516129   3.627734  ...  66.0  67.00   76.0
           6      227.0  70.127753   4.200345  ...  69.0  72.00   82.0
           7      568.0  77.966549   5.038190  ...  78.0  81.00   92.0
           8     1206.0  86.549751   5.454034  ...  87.0  90.00   99.0
           9     1491.0  89.429242   5.140919  ...  91.0  92.00  100.0
           10    1461.0  88.824093   4.702797  ...  90.0  92.00  100.0
           11    1364.0  87.535924   4.282507  ...  89.0  90.00   98.0
           12    1265.0  86.233992   3.919759  ...  87.0  89.00   96.0
           13    1174.0  84.952300   3.654439  ...  86.0  87.00   94.0
           14    1092.0  83.767399   3.347971  ...  84.0  86.00   92.0
           15    1003.0  82.681954   3.101983  ...  83.0  84.00   90.0
           16     918.0  81.601307   2.944429  ...  82.0  83.00   88.0
           17     827.0  80.366385   2.797436  ...  81.0  82.00   87.0
           18     786.0  79.977099   4.926412  ...  79.0  81.00  100.0
           19     704.0  79.244318   6.113900  ...  78.0  79.00   98.0
           20     602.0  77.755814   5.916938  ...  76.0  78.00   96.0
           21     514.0  76.365759   5.703738  ...  75.0  76.00   93.0
           22     445.0  74.901124   5.342682  ...  74.0  75.00   90.0
           23     376.0  73.590426   5.134699  ...  72.0  74.00   88.0
photon_002 0      270.0  61.662963  29.073572  ...  74.0  86.00   94.0
           1      268.0  60.347015  29.099257  ...  72.0  85.00   93.0
           2      257.0  61.821012  26.621608  ...  72.0  84.00   91.0
           3      257.0  60.727626  26.348693  ...  71.0  83.00   90.0
           4      254.0  59.677165  26.482564  ...  70.0  82.00   89.0
           5      289.0  52.335747  29.941125  ...  59.0  80.00   88.0
           6      347.0  46.267080  32.333627  ...  47.0  81.00   91.0
           7      354.0  47.709047  33.799798  ...  46.0  84.00   95.0
           8      362.0  51.323599  35.958791  ...  51.0  89.00  100.0
           9      355.0  52.765178  37.011403  ...  50.0  92.00  100.0
           10     356.0  53.568319  37.215838  ...  52.5  93.25  100.0
           11     358.0  53.496342  37.257862  ...  52.5  95.25  100.0
           12     354.0  52.936998  37.345374  ...  52.5  93.00  100.0
           13     353.0  52.672557  37.368148  ...  53.0  93.00  100.0
           14     352.0  52.214240  37.332135  ...  53.0  93.50   99.0
           15     358.0  51.926930  37.430981  ...  53.0  93.00   99.0
           16     358.0  50.784788  37.492062  ...  52.0  93.00   99.0
           17     359.0  50.581691  37.475200  ...  52.0  92.50   98.0
           18     360.0  50.548085  37.701102  ...  51.5  92.00  100.0
           19     347.0  52.848801  37.146308  ...  57.0  91.50   99.0
           20     329.0  54.157868  36.379758  ...  60.0  90.00   98.0
           21     302.0  57.338671  34.222796  ...  68.0  89.00   97.0
           22     273.0  61.249069  30.950664  ...  73.0  89.00   96.0
           23     279.0  60.303668  31.002111  ...  72.0  87.00   95.0

[48 rows x 8 columns]

In [56]:
obs_hour = hourly_stats.reset_index()
obs_hour.columns = [column[0] if column[1] == '' else column[1] for column in obs_hour.columns]
days = (sbs_df['timestamp'].max() - sbs_df['timestamp'].min()).days
obs_hour['avg_count'] = obs_hour['count'].apply(lambda count: count / days)
obs_hour[['device_id', 'avg_count']].groupby(['device_id']).describe()

avg_count                        ...                                 
               count       mean        std  ...        50%        75%        max
device_id                                   ...                                 
photon_001      24.0  21.977273  13.471992  ...  19.787879  33.712121  45.181818
photon_002      24.0   9.786616   1.266327  ...  10.590909  10.803030  10.969697

[2 rows x 8 columns]

In [88]:
fig3 = go.Figure([
                  go.Bar(
                      x=obs_hour[obs_hour['device_id'] == 'photon_001']['hour'],
                      y=obs_hour[obs_hour['device_id'] == 'photon_001']['avg_count'],
                      legendgroup="photon001",
                      name='Device w/ Hypnos'
                  ),
                  go.Bar(
                      x=obs_hour[obs_hour['device_id'] == 'photon_002']['hour'],
                      y=obs_hour[obs_hour['device_id'] == 'photon_002']['avg_count'],
                      legendgroup="photon002",
                      name='Device w/o Hypnos'
                  ),
                  go.Scatter(
                      x=obs_hour[obs_hour['device_id'] == 'photon_001']['hour'],
                      y=obs_hour[obs_hour['device_id'] == 'photon_001']['mean'],
                      mode='lines', 
                      legendgroup="photon001",
                      name='Device w/ Hypnos - avg. battery (%)',
                      line=dict(color=px.colors.qualitative.Plotly[0])
                  ),
                  go.Scatter(
                      x=obs_hour[obs_hour['device_id'] == 'photon_002']['hour'],
                      y=obs_hour[obs_hour['device_id'] == 'photon_002']['mean'],
                      legendgroup="photon002",
                      mode='lines', 
                      name='Device w/o Hypnos - avg. battery (%)',
                      line=dict(color=px.colors.qualitative.Plotly[1])
                  )
])
fig3.update_layout(barmode='group')

fig3.update_layout(
    xaxis_title="Hour of the day", 
    yaxis_title="Observation count (avg.)")

style_legend(fig3)

fig3.show()

In [58]:
bat_obs = df[df['type'] == 'battery']
bat_obs[['device_id', 'day', 'value']].groupby(['device_id', 'day']).describe()

value             ...                       
                       count       mean  ...        75%         max
device_id  day                           ...                       
photon_001 2017-04-30  308.0  92.964286  ...  97.000000  100.000000
           2017-05-01  950.0  88.858947  ...  96.000000  100.000000
           2017-05-02  374.0  82.973262  ...  94.000000   94.000000
           2017-05-03  563.0  81.779751  ...  86.000000   87.000000
           2017-05-04  642.0  84.389408  ...  90.000000   92.000000
...                      ...        ...  ...        ...         ...
photon_002 2017-05-30  179.0   8.056986  ...   9.811176    9.811176
           2017-05-31  179.0   5.883746  ...   7.414472    7.414472
           2017-06-01  162.0   4.893880  ...   6.838324    6.838324
           2017-06-02   10.0   0.312911  ...   0.312911    0.312911
           2017-06-03    1.0  -0.398583  ...  -0.398583   -0.398583

[108 rows x 8 columns]

In [89]:
daily_count = bat_obs[['device_id', 'day', 'value']].groupby(['device_id', 'day']).count().reset_index()
fig4 = px.bar(daily_count, x='day', y='value', color='device_id', barmode='group')

fig4.update_layout(
    xaxis_title="Date", 
    yaxis_title="Observation count")

update_device_names(fig4)
style_legend(fig4)

remove_year(fig4)

fig4.show()

In [60]:
daily_obs = bat_obs[['device_id', 'day', 'value']].groupby(['device_id', 'day']).describe().reset_index()
daily_obs.columns = [column[0] if column[1] == '' else column[1] for column in daily_obs.columns]
daily_obs = daily_obs[['device_id', 'day', 'count']]

exp_daily_obs = 24 * 60 / 5
daily_obs.groupby(['device_id']).describe()

count                                                          
           count        mean         std   min    25%    50%    75%    max
device_id                                                                 
photon_001  73.0  468.246575  169.247582  33.0  368.0  505.0  590.0  950.0
photon_002  35.0  221.457143   75.322968   1.0  180.5  267.0  267.0  268.0

In [61]:
import plotly.figure_factory as ff

fig6 = ff.create_distplot([
                           daily_obs[daily_obs['device_id'] == 'photon_001']['count'],
                           daily_obs[daily_obs['device_id'] == 'photon_002']['count']
], ['Device w/ Hypnos', 'Device w/o Hypnos'], colors=px.colors.qualitative.Plotly)

fig6.update_layout(
    xaxis_title="Daily observations", 
    yaxis_title="Probability")

style_legend(fig6)
fig6.update_layout(legend=dict(
    traceorder='normal'
))

fig6.show()

In [62]:
daily_obs[daily_obs['count'] < 267].groupby(['device_id']).describe()

count                                                     
           count   mean        std   min     25%    50%    75%    max
device_id                                                            
photon_001  10.0  159.2  71.515033  33.0  116.25  162.5  214.0  261.0
photon_002  15.0  160.6  82.669566   1.0  140.00  179.0  196.5  266.0